In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# AdaBoost Algorithm
from sklearn.ensemble import AdaBoostClassifier
# Gradient Boosting 
from sklearn.ensemble import GradientBoostingClassifier
# XGBoost 
import xgboost
from xgboost import XGBClassifier
from xgboost import plot_importance, to_graphviz, plot_tree
print("XGBoost version:",xgboost.__version__)
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, cross_val_score

mycmap = "winter"
mpl.rcParams['image.cmap'] = mycmap
plt.rcParams['font.size'] = 13

XGBoost version: 2.0.3


2024-03-21 09:11:47.732772: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 09:11:47.770102: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-21 09:11:47.770137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-21 09:11:47.772057: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-21 09:11:47.777838: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 09:11:47.778587: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
np.random.seed(12345)

dname="./DATA/"
str0="_XGB_24.dat"
fnamex=dname+'x'+str0
fnamey=dname+'y'+str0
x = np.loadtxt(fnamex, delimiter=" ",dtype=float)
y = np.loadtxt(fnamey)
y = y.astype(int)
N,L = len(x), len(x[0])


In [3]:
num_neurons=20
num_layers=2
activation_function='relu'
perc=0.7
n_rep=30

In [5]:
def build_nn(num_layers,num_neurons,activation_function):
    input_shape = x.shape[1]
    model_nn = Sequential()
    model_nn.add(Dense(units=num_neurons, activation=activation_function, input_shape=(input_shape,)))
    for i in range(num_layers-1):
        model_nn.add(Dense(units=num_neurons, activation=activation_function))
    model_nn.add(Dense(units=2,activation='softmax'))
    opt = keras.optimizers.Adam()
    model_nn.compile(loss=keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])
    return model_nn

num_layers=[2,3,4,5,6,7,8,9,10]                      #num of nn layers(input and output excluded)
num_neurons=[5,10,15,20,25,30,35,40]                        #num of neurons per layer
activation_functions=['relu']         #activation function for the hidden layers
max_acc=0
max_desc=""
perc=0.7
n_rep=20
num_indices=int(perc*len(x))        #get quantity of data points in training
data_indices = np.arange(len(x))    #get indices
mean_acc_list=[]

for nl in num_layers:
    for nn in num_neurons:
        for act in activation_functions:
            accuracies_nn=[]
            print(f"--RUNNING--n_lay: {nl}, n_neu: {nn}, act: {act}")
            for i in range(n_rep):
                np.random.shuffle(data_indices)     #shuffle indices
                train_indices = data_indices[:num_indices]  #get training set indices
                test_indices = data_indices[num_indices:]   #get test set indices
                
                x_train, y_train = x[train_indices], y[train_indices]
                x_test, y_test = x[test_indices], y[test_indices]
                y_train = keras.utils.to_categorical(y_train, num_classes=2)
                y_test = keras.utils.to_categorical(y_test, num_classes=2)
                
                
                model=build_nn(num_layers=nl,num_neurons=nn,activation_function=act)
                model.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=0)
                accuracy_nn = model.evaluate(x_test, y_test, verbose=0)[1]  #accuracy is at index 1
                accuracies_nn.append(accuracy_nn)
                #print(accuracy_nn)
                del model,x_train,x_test,y_train,y_test      #just to free memory
            mean_acc=np.mean(accuracies_nn)
            mean_acc_list.append(mean_acc)
            print(mean_acc)
            if(mean_acc>max_acc):
                max_acc = mean_acc
                max_desc = f"n_lay: {nl}, n_neu: {nn}, act: {act}"
print(f"Best mean accuracy ({max_acc}) obtained by the model with parameters: {max_desc}")

2800
1200


2024-03-21 09:11:49.506232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-21 09:11:49.506957: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


0.5216666460037231
2800
1200
0.5483333468437195
2800
1200
0.5116666555404663
2800
1200
0.550000011920929
2800
1200
0.51583331823349
2800
1200
0.51583331823349
2800
1200
0.51583331823349
2800
1200
0.5241666436195374
2800
1200
0.5049999952316284
2800
1200
0.5350000262260437
2800
1200
0.5408333539962769
2800
1200
0.5366666913032532
2800
1200
0.5383333563804626
2800
1200
0.5733333230018616
2800
1200
0.5425000190734863
2800
1200
0.5233333110809326
2800
1200
0.48249998688697815
2800
1200
0.528333306312561
2800
1200
0.5691666603088379
2800
1200
0.5458333492279053
2800
1200
0.559166669845581
2800
1200
0.5299999713897705
2800
1200
0.550000011920929
2800
1200
0.5291666388511658
2800
1200
0.5533333420753479
2800
1200
0.5233333110809326
2800
1200
0.5391666889190674
2800
1200
0.5366666913032532
2800
1200
0.5358333587646484
2800
1200
0.5350000262260437
0.5338611116011938


In [6]:

accuracies_nn=[]

for i in range(n_rep):
    num_indices=int(perc*len(x))        #get quantity of data points in training
    data_indices = np.arange(len(x))    #get indices
    np.random.shuffle(data_indices)     #shuffle indices
    train_indices = data_indices[:num_indices]  #get training set indices
    test_indices = data_indices[num_indices:]   #get test set indices
    
    x_train, y_train = x[train_indices], y[train_indices]
    x_test, y_test = x[test_indices], y[test_indices]
    y_train = keras.utils.to_categorical(y_train, num_classes=2)
    y_test = keras.utils.to_categorical(y_test, num_classes=2)
    print(len(x_train))
    print(len(x_test))
    
    model_nn=build_model()
    #print(f"--RUNNING ({i+1}/{n_rep})--n_lay: {nl}, n_neu: {nn}, act: {act}")
    model_nn.fit(x_train, y_train, validation_data=(x_test, y_test), verbose=0)
    accuracy_nn = model_nn.evaluate(x_test, y_test, verbose=0)[1]  #accuracy is at index 1
    accuracies_nn.append(accuracy_nn)
    print(accuracy_nn)
print(np.mean(accuracies_nn))

2800
1200
0.5308333039283752
2800
1200
0.5699999928474426
2800
1200
0.5649999976158142
2800
1200
0.5358333587646484
2800
1200
0.5633333325386047
2800
1200
0.5625
2800
1200
0.51583331823349
2800
1200
0.5516666769981384
2800
1200
0.5191666483879089
2800
1200
0.5241666436195374
2800
1200
0.5149999856948853
2800
1200
0.5408333539962769
2800
1200
0.5416666865348816
2800
1200
0.5241666436195374
2800
1200
0.54666668176651
2800
1200
0.528333306312561
2800
1200
